In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['test.csv', 'train.csv', 'sample_submission.csv', 'data_description.txt']


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer

data = pd.read_csv('../input/train.csv')
#data.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = data['SalePrice']
X = data.drop(['SalePrice'], axis=1).select_dtypes(exclude=['object'])


my_imputer = Imputer()
Imp_X = my_imputer.fit_transform(X)
#Imp_y = my_imputer.fit_transform(y)
#test_X = my_imputer.transform(test_X)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [3]:
from sklearn import preprocessing
X_scaled = preprocessing.scale(Imp_X)
y_scaled = preprocessing.scale (y)

/opt/conda/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [4]:
from sklearn import linear_model
param_alpha= 0.2
colnames = list (data)
#y = data['SalePrice']
#predictors = [x for x in colnames if x!='SalePrice'] 
lassoreg = linear_model.Lasso(alpha = param_alpha)
lassoreg.fit(X_scaled, y_scaled) 

Lasso(alpha=0.2, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [5]:
lassoreg.coef_
coefs_lst = list(lassoreg.coef_)
mask = list(map(lambda x: x != 0, coefs_lst))
important_vars = []
for x,z in enumerate(list(X)):
    if mask[x]:
        important_vars.append(z)

In [6]:
important_vars

['OverallQual',
 'TotalBsmtSF',
 '1stFlrSF',
 'GrLivArea',
 'GarageCars',
 'GarageArea']

In [7]:
type(important_vars)

list

In [8]:
X_important = data[important_vars]
X_important.head()

,OverallQual,TotalBsmtSF,1stFlrSF,GrLivArea,GarageCars,GarageArea
0,7,856,856,1710,2,548
1,6,1262,1262,1262,2,460
2,7,920,920,1786,2,608
3,7,756,961,1717,3,642
4,8,1145,1145,2198,3,836


In [9]:
imptnt_train_X, imptnt_test_X, imptnt_train_y, imptnt_test_y = train_test_split(X_important, y, test_size=0.25)

In [10]:
my_model_important = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model_important.fit(imptnt_train_X, imptnt_train_y, early_stopping_rounds=5, 
             eval_set=[(imptnt_test_X, imptnt_test_y)], verbose=False)
predictions_imptnt = my_model_important.predict(imptnt_test_X)
from sklearn.metrics import mean_absolute_error
print("Mean Absolute Error : " + str(mean_absolute_error(predictions_imptnt, imptnt_test_y)))

NameError: name 'XGBRegressor' is not defined